In [1]:
import os

In [2]:
%pwd

'/Users/harshaabhinavkusampudi/Documents/Text_Summarizer/Text_Summarizer_Project/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/harshaabhinavkusampudi/Documents/Text_Summarizer/Text_Summarizer_Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=2)
class DataIngestionConfig:
    root_dir:Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    


In [6]:
!which python


/Users/harshaabhinavkusampudi/miniconda3/envs/text5/bin/python


In [7]:
!pip show TextSummarizer


Name: TextSummarizer
Version: 0.0.1
Summary: A small python package for NLP app
Home-page: https://github.com/Harshabhi6129/Text_Summarizer_Project
Author: Harshabhi6129
Author-email: harshakusampudi@gmail.com
License: 
Location: /Users/harshaabhinavkusampudi/Documents/Text_Summarizer/Text_Summarizer_Project/src
Editable project location: /Users/harshaabhinavkusampudi/Documents/Text_Summarizer/Text_Summarizer_Project/src
Requires: 
Required-by: 


In [8]:
!pip install -e .


Obtaining file:///Users/harshaabhinavkusampudi/Documents/Text_Summarizer/Text_Summarizer_Project
  Preparing metadata (setup.py) ... done
  Attempting uninstall: TextSummarizer
    Found existing installation: TextSummarizer 0.0.1
    Uninstalling TextSummarizer-0.0.1:
      Successfully uninstalled TextSummarizer-0.0.1
  DEPRECATION: Legacy editable install of TextSummarizer==0.0.1 from file:///Users/harshaabhinavkusampudi/Documents/Text_Summarizer/Text_Summarizer_Project (setup.py develop) is deprecated. pip 25.0 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457
  Running setup.py develop for TextSummarizer


In [9]:
!pip install python-box


In [10]:
!pip install PyYAML


In [11]:
!pip install ensure


In [12]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories



✅ Config file path resolved to: /Users/harshaabhinavkusampudi/Documents/Text_Summarizer/Text_Summarizer_Project/config/config.yaml


In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [14]:
import os
import urllib.request as request
import zipfile
from TextSummarizer.logging import logger
from TextSummarizer.utils.common import get_size

In [15]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [16]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

🔍 Trying to open YAML file at: /Users/harshaabhinavkusampudi/Documents/Text_Summarizer/Text_Summarizer_Project/config/config.yaml
✅ YAML file loaded successfully!
🔍 Trying to open YAML file at: /Users/harshaabhinavkusampudi/Documents/Text_Summarizer/Text_Summarizer_Project/params.yaml
✅ YAML file loaded successfully!
[2025-03-21 14:52:12,090: INFO: common: created directory at: artifacts]
[2025-03-21 14:52:12,091: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-21 14:52:12,998: INFO: 1434958058: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: EC9F:38